In [2]:
from pyrosetta import *
from pyrosetta.rosetta import *
from MiningMinima import *

PyRosetta-4 2019 [Rosetta PyRosetta4.Release.python27.ubuntu 2019.33+release.1e60c63beb532fd475f0f704d68d462b8af2a977 2019-08-09T15:19:57] retrieved from: http://www.pyrosetta.org
(C) Copyright Rosetta Commons Member Institutions. Created in JHU by Sergey Lyskov and PyRosetta Team.
core.init: Checking for fconfig files in pwd and ./rosetta/flags
core.init: Rosetta version: PyRosetta4.Release.python27.ubuntu r230 2019.33+release.1e60c63beb5 1e60c63beb532fd475f0f704d68d462b8af2a977 http://www.pyrosetta.org 2019-08-09T15:19:57
core.init: command: PyRosetta -ex1 -ex2aro -database /usr/local/lib/python2.7/dist-packages/pyrosetta-2019.33+release.1e60c63beb5-py2.7-linux-x86_64.egg/pyrosetta/database
basic.random.init_random_generator: 'RNG device' seed mode, using '/dev/urandom', seed=-128115017 seed_offset=0 real_seed=-128115017
basic.random.init_random_generator: RandomGenerator:init: Normal mode, seed=-128115017 RG_type=mt19937


# Developing subroutines to integrate over rigid body dofs

## Initialize stuff

In [59]:
seq1, seq2 = 'a', 'u'
pose = protocols.recces.pose_setup_turner(seq1, seq2)
n_res = len(seq1) + len(seq2)

# set fold tree using usual chi torsion atoms
ft = FoldTree(n_res)
ft.new_jump(1, n_res, n_res/2)
ft.set_jump_atoms(1, pose.residue(1).atom_name(pose.residue(1).chi_atoms(1)[4]),
    pose.residue(n_res).atom_name(pose.residue(n_res).chi_atoms(1)[4]))
pose.fold_tree(ft)

# set jump dofs in movemap
movemap = MoveMap
movemap.set_jump(1, True)

# get scorefxn
scorefxn = core.scoring.ScoreFunctionFactory.create_score_function('stepwise/rna/turner_new')

In [60]:
pmm = PyMOLMover()
pmm.apply(pose)

## Set up multifunc and minimize

In [61]:
# taken from MiningMinima class
def set_up_multifunc(pose, movemap, scorefxn):
    # set up minimizer map
    min_map = core.optimization.MinimizerMap()
    min_map.setup(pose, movemap)

    start_score = scorefxn(pose)
    pose.energies().set_use_nblist(pose, min_map.domain_map(), True)
    multifunc = core.optimization.AtomTreeMultifunc(pose, min_map, scorefxn)
    scorefxn.setup_for_minimizing(pose, min_map)
    scorefxn.setup_for_derivatives(pose)

    min_map = min_map
    multifunc = multifunc
    
    return multifunc, min_map

In [62]:
multifunc, min_map = set_up_multifunc(pose, movemap, scorefxn)
min_dofs = Vector1([0.0]*min_map.nangles())
min_map.copy_dofs_from_pose(pose,min_dofs)
start_func_val = multifunc(min_dofs)
theta = Vector1(list(min_dofs))
dE_dtheta = Vector1(list(min_dofs))
multifunc.dfunc(theta,dE_dtheta)

In [64]:
min_options = core.optimization.MinimizerOptions(
    'lbfgs_armijo_nonmonotone', 1e-15, True, False, False)
min_options.nblist_auto_update(True)
minimizer = core.optimization.Minimizer(multifunc, min_options)

In [65]:
min_dofs = Vector1(list(theta))
minimizer.run(min_dofs)

-11.468164977405111

In [66]:
min_map.copy_dofs_to_pose(pose, min_dofs)
pmm.apply(pose)